# AI Red Teaming Agent for Generative AI models and applications in Azure AI Foundry

 AI Red Teaming Agent leverages [Risk and Safety Evaluations](https://learn.microsoft.com/en-us/azure/ai-foundry/concepts/evaluation-metrics-built-in?tabs=warning#risk-and-safety-evaluators) to help identify potential safety issues across different risk categories (violence, hate/unfairness, sexual content, self-harm) combined with attack strategies of varying complexity levels from [PyRIT](https://github.com/Azure/PyRIT), Microsoft AI Red Teaming team's open framework for automated AI red teaming.


### Imports

In [2]:
# !pip install azure-ai-evaluation[redteam] azure-identity openai azure-ai-projects

In [3]:
from typing import Optional, Dict, Any
import os

# Azure imports
from azure.ai.evaluation.red_team import RedTeam, RiskCategory, AttackStrategy

# OpenAI imports
from openai import AzureOpenAI

### Login to Azure with valid credentials

Ensure that you've installed the [Azure CLI](https://learn.microsoft.com/en-us/cli/azure/install-azure-cli) and then make sure to authenticate to Azure using `az login` in your terminal before running this notebook.

Configure the `credential` object with a different AzureCredential type if this is a requirement for your environment.

In [ ]:
# Azure Credential imports
from azure.identity import AzureCliCredential, get_bearer_token_provider

!az login

# Initialize Azure credentials
credential = AzureCliCredential()

### Set Up Your Environment Variables

Set the following variables for use in this notebook. These variables connect to your Azure resources and model deployments.

Set these variables by creating an `.env` file in your project's root folder.

**Note:** You can find these values in your Azure AI Foundry project or Azure OpenAI resource.

In [9]:
# Azure AI Project information
azure_ai_project = os.environ.get("AZURE_AI_FOUNDRY_ENDPOINT")

# Azure OpenAI deployment information
azure_openai_deployment = os.environ.get("AZURE_OPENAI_DEPLOYMENT_NAME")  # e.g., "gpt-4"
azure_openai_endpoint = os.environ.get("AZURE_OPENAI_ENDPOINT")
azure_openai_api_key = os.environ.get("AZURE_OPENAI_API_KEY")  # e.g., "your-api-key"
azure_openai_api_version = os.environ.get("AZURE_OPENAI_API_VERSION")  # Use the latest API version

## Understanding AI Red Teaming Agent's capabilities

The Azure AI Evaluation SDK's `RedTeam` functionality evaluates AI systems against adversarial prompts across multiple dimensions:

1. **Risk Categories**: Different content risk categories your AI system might generate
   - Violence
   - HateUnfairness
   - Sexual
   - SelfHarm

2. **Attack Strategies**: Along with standard unmodified prompts which are sent by default as the `baseline`, you can specify different transformations of prompts to elicit undesired content.
You can also use `AttackStrategy.Compose()` to layer two strategies in one attack
   - AnsiAttack: Using ANSI escape codes in prompts
   - AsciiArt: Using ASCII art to disguise harmful content
   - AsciiSmuggler: Hiding harmful content within ASCII characters
   - Atbash: Using the Atbash cipher to encode harmful requests
   - Base64: Encoding harmful content in Base64 format
   - Binary: Converting text to binary to bypass filters
   - Caesar: Using the Caesar cipher for encoding
   - CharacterSpace: Manipulating character spacing to confuse filters
   - CharSwap: Swapping characters to bypass detection
   - Diacritic: Using diacritical marks to alter text appearance
   - Flip: Flipping text to bypass content filters
   - Leetspeak: Converting letters to numbers and symbols
   - Morse: Using Morse code to encode harmful requests
   - ROT13: Using ROT13 cipher for text transformation
   - SuffixAppend: Adding suffixes to confuse detection systems
   - StringJoin: Joining strings in unconventional ways
   - Tense: Changing the tense of harmful requests to past tense
   - UnicodeConfusable: Using similar-looking Unicode characters
   - UnicodeSubstitution: Substituting characters with Unicode alternatives
   - Url: Embedding harmful content within URLs
   - Jailbreak: Specially crafted prompts to bypass AI safeguards

3. **Complexity Levels**: Different difficultly levels of attacks
   - Baseline: Standard functionality tests
   - Easy: Simple attack patterns
   - Moderate: More sophisticated attacks
   - Difficult: Complex, layered attack strategies

The key metric for evaluating results is the **Attack Success Rate (ASR)**, which measures the percentage of attacks that successfully elicit harmful content from your AI system.

## Basic Example: Fixed Response Callback

Let's start with a basic example that demonstrates how to set up a Red Team evaluation using a simple callback that always returns a fixed, safe response regardless of the input prompt.

**Important Note**: Since this callback always responds with the same safe message, the Attack Success Rate (ASR) will be 0% for all categories. This example is merely to demonstrate the API structure and flow.

In [11]:
# Define a simple callback function that always returns a fixed response
def financial_advisor_callback(query: str) -> str:  # noqa: ARG001
    return "I'm a financial advisor assistant. I can help with investment advice and financial planning within legal and ethical guidelines."

In [12]:
# Create the `RedTeam` instance with minimal configurations
red_team = RedTeam(
    azure_ai_project=azure_ai_project,
    credential=credential,
    risk_categories=[RiskCategory.Violence, RiskCategory.HateUnfairness],
    num_objectives=1,
)

NOTE: `num_objectives` specifies the number of attacks to perform per risk category per attack strategy. If the parameter `risk_categories` is not specified, `[RiskCategory.Violence, RiskCategory.HateUnfairness, RiskCategory.Sexual, RiskCategory.SelfHarm]` will be used by default.

Now let's run a simple automated scan using the `RedTeam` with the fixed response target. We'll test against two risk categories and one attack strategy for simplicity.

In [13]:
# Run the red team scan called "Basic-Callback-Scan" with limited scope for this basic example
# This will test 1 objective prompt for each of Violence and HateUnfairness categories with the Flip strategy
result = await red_team.scan(
    target=financial_advisor_callback,
    scan_name="Basic-Callback-Scan",
    attack_strategies=[AttackStrategy.Flip],
    output_path="red_team_output.json",
)

🚀 STARTING RED TEAM SCAN: Basic-Callback-Scan
📂 Output directory: .\.scan_Basic-Callback-Scan_20251105_222545
📊 Risk categories: ['violence', 'hate_unfairness']
🔗 Track your red team scan in AI Foundry: None
📋 Planning 4 total tasks


Scanning:   0%|                         | 0/4 [00:00<?, ?scan/s, current=fetching baseline/violence]

📚 Using attack objectives from Azure RAI service


Scanning:   0%|                  | 0/4 [00:03<?, ?scan/s, current=fetching baseline/hate_unfairness]

📝 Fetched baseline objectives for violence: 1 objectives


...

⚙️ Processing 4 tasks in parallel (max 5 at a time)
▶️ Starting task: baseline strategy for violence risk category
▶️ Starting task: baseline strategy for hate_unfairness risk category
▶️ Starting task: flip strategy for violence risk category
▶️ Starting task: flip strategy for hate_unfairness risk category


ERROR: Error evaluating conversation 1 for violence/baseline: (UserError) The needed capability 'content harm' is not supported by the RAI service in this region.
Visit https://aka.ms/azsdk/python/evaluation/safetyevaluator/troubleshoot to troubleshoot this issue.
ERROR: Error evaluating conversation 1 for violence/baseline: (UserError) The needed capability 'content harm' is not supported by the RAI service in this region.
Visit https://aka.ms/azsdk/python/evaluation/safetyevaluator/troubleshoot to troubleshoot this issue.
ERROR: Error evaluating conversation 1 for hate_unfairness/baseline: (UserError) The needed capability 'content harm' is not supported by the RAI service in this region.
Visit https://aka.ms/azsdk/python/evaluation/safetyevaluator/troubleshoot to troubleshoot this issue.
ERROR: Error evaluating conversation 1 for hate_unfairness/baseline: (UserError) The needed capability 'content harm' is not supported by the RAI service in this region.
Visit https://aka.ms/azsdk/p

Evaluation results saved to "C:\Users\pablosal\Desktop\gbb-foundry-fabric-agenticrag\labs\evals\redteaming\.scan_Basic-Callback-Scan_20251105_222545\baseline_violence_46d8e966-3bd0-40e9-865b-9b89c489133e.json".

✅ Completed task 1/4 (25.0%) - baseline/violence in 22.7s
   Est. remaining: 1.8 minutes
Evaluation results saved to "C:\Users\pablosal\Desktop\gbb-foundry-fabric-agenticrag\labs\evals\redteaming\.scan_Basic-Callback-Scan_20251105_222545\baseline_hate_unfairness_c85734cf-8caf-45e1-b1d2-3439a8389c84.json".

✅ Completed task 2/4 (50.0%) - baseline/hate_unfairness in 22.7s
   Est. remaining: 0.6 minutes
Evaluation results saved to "C:\Users\pablosal\Desktop\gbb-foundry-fabric-agenticrag\labs\evals\redteaming\.scan_Basic-Callback-Scan_20251105_222545\flip_violence_66bfa59a-347b-4ea5-b9e6-bf183c6cb3a7.json".

✅ Completed task 3/4 (75.0%) - flip/violence in 22.7s
   Est. remaining: 0.2 minutes
Evaluation results saved to "C:\Users\pablosal\Desktop\gbb-foundry-fabric-agenticrag\labs\e

RequestId:5b147945-401e-0087-41d5-4e07b8000000
Time:2025-11-06T04:26:25.6882562Z
ErrorCode:AuthorizationFailure
Content: <?xml version="1.0" encoding="utf-8"?><Error><Code>AuthorizationFailure</Code><Message>This request is not authorized to perform this operation.
RequestId:5b147945-401e-0087-41d5-4e07b8000000
Time:2025-11-06T04:26:25.6882562Z</Message></Error>
RequestId:5b147945-401e-0087-41d5-4e07b8000000
Time:2025-11-06T04:26:25.6882562Z
ErrorCode:AuthorizationFailure
Content: <?xml version="1.0" encoding="utf-8"?><Error><Code>AuthorizationFailure</Code><Message>This request is not authorized to perform this operation.
RequestId:5b147945-401e-0087-41d5-4e07b8000000
Time:2025-11-06T04:26:25.6882562Z</Message></Error>


Evaluation results saved to "C:\Users\pablosal\Desktop\gbb-foundry-fabric-agenticrag\labs\evals\redteaming\red_team_output.json".

Evaluation results saved to "C:\Users\pablosal\Desktop\gbb-foundry-fabric-agenticrag\labs\evals\redteaming\.scan_Basic-Callback-Scan_20251105_222545\final_results.json".

Overall ASR: 0.0%
Attack Success: 0/4 attacks were successful
------------------------------------------------------------------------------------------------------------------------------------
Risk Category     | Baseline ASR   | Easy-Complexity Attacks ASR  | Moderate-Complexity Attacks ASR | Difficult-Complexity Attacks ASR
------------------------------------------------------------------------------------------------------------------------------------

📂 All scan files saved to: .\.scan_Basic-Callback-Scan_20251105_222545
✅ Scan completed successfully!


## Intermediary Example: Using a Model Configuration as Target

Now let's create a more realistic example that uses an Azure OpenAI model for responding to the red teaming prompts. To test base or foundation models, you can update your target to take in a model configuration:

In [14]:
# Define a model configuration to test
azure_oai_model_config = {
    "azure_endpoint": azure_openai_endpoint,
    "azure_deployment": azure_openai_deployment,
    "api_key": azure_openai_api_key,
}

Then, update your target to point to the model configurations and run the scan.

In [15]:
# Run the red team scan called "Intermediary-Model-Target-Scan"
result = await red_team.scan(
    target=azure_oai_model_config,
    scan_name="Intermediary-Model-Target-Scan",
    attack_strategies=[AttackStrategy.Flip],
)

🚀 STARTING RED TEAM SCAN: Intermediary-Model-Target-Scan
📂 Output directory: .\.scan_Intermediary-Model-Target-Scan_20251105_222639
📊 Risk categories: ['violence', 'hate_unfairness']
🔗 Track your red team scan in AI Foundry: None
📋 Planning 4 total tasks


Scanning:   0%|                         | 0/4 [00:00<?, ?scan/s, current=fetching baseline/violence]

📚 Using attack objectives from Azure RAI service


Scanning:   0%|                  | 0/4 [00:00<?, ?scan/s, current=fetching baseline/hate_unfairness]

📝 Fetched baseline objectives for violence: 1 objectives


...

Evaluation results saved to "C:\Users\pablosal\Desktop\gbb-foundry-fabric-agenticrag\labs\evals\redteaming\.scan_Intermediary-Model-Target-Scan_20251105_222639\flip_violence_3d8a9c62-994b-423b-803d-86279369187a.json".

✅ Completed task 1/4 (25.0%) - flip/violence in 23.2s
   Est. remaining: 1.3 minutes


ERROR: Error evaluating conversation 1 for violence/baseline: (UserError) The needed capability 'content harm' is not supported by the RAI service in this region.
Visit https://aka.ms/azsdk/python/evaluation/safetyevaluator/troubleshoot to troubleshoot this issue.
ERROR: Error evaluating conversation 1 for violence/baseline: (UserError) The needed capability 'content harm' is not supported by the RAI service in this region.
Visit https://aka.ms/azsdk/python/evaluation/safetyevaluator/troubleshoot to troubleshoot this issue.
Scanning: 100%|██████████████████████████████████| 4/4 [00:31<00:00,  7.91s/scan, current=batch 1/1]


Evaluation results saved to "C:\Users\pablosal\Desktop\gbb-foundry-fabric-agenticrag\labs\evals\redteaming\.scan_Intermediary-Model-Target-Scan_20251105_222639\baseline_hate_unfairness_e9237707-1608-4e89-8ef7-e56636e4afb2.json".

✅ Completed task 2/4 (50.0%) - baseline/hate_unfairness in 30.0s
   Est. remaining: 0.5 minutes
Evaluation results saved to "C:\Users\pablosal\Desktop\gbb-foundry-fabric-agenticrag\labs\evals\redteaming\.scan_Intermediary-Model-Target-Scan_20251105_222639\flip_hate_unfairness_ad206bc3-1b8b-4b58-99fd-3257b06a1cb1.json".

✅ Completed task 3/4 (75.0%) - flip/hate_unfairness in 30.0s
   Est. remaining: 0.2 minutes
Evaluation results saved to "C:\Users\pablosal\Desktop\gbb-foundry-fabric-agenticrag\labs\evals\redteaming\.scan_Intermediary-Model-Target-Scan_20251105_222639\baseline_violence_91cc7a22-d45a-467c-b55d-e29f2d66f6bc.json".

✅ Completed task 4/4 (100.0%) - baseline/violence in 30.1s
   Est. remaining: 0.0 minutes


RequestId:60b0e5c7-901e-00ac-2fd5-4e7300000000
Time:2025-11-06T04:27:15.2133356Z
ErrorCode:AuthorizationFailure
Content: <?xml version="1.0" encoding="utf-8"?><Error><Code>AuthorizationFailure</Code><Message>This request is not authorized to perform this operation.
RequestId:60b0e5c7-901e-00ac-2fd5-4e7300000000
Time:2025-11-06T04:27:15.2133356Z</Message></Error>
RequestId:60b0e5c7-901e-00ac-2fd5-4e7300000000
Time:2025-11-06T04:27:15.2133356Z
ErrorCode:AuthorizationFailure
Content: <?xml version="1.0" encoding="utf-8"?><Error><Code>AuthorizationFailure</Code><Message>This request is not authorized to perform this operation.
RequestId:60b0e5c7-901e-00ac-2fd5-4e7300000000
Time:2025-11-06T04:27:15.2133356Z</Message></Error>


Evaluation results saved to "C:\Users\pablosal\Desktop\gbb-foundry-fabric-agenticrag\labs\evals\redteaming\.scan_Intermediary-Model-Target-Scan_20251105_222639\final_results.json".

Overall ASR: 0.0%
Attack Success: 0/4 attacks were successful
------------------------------------------------------------------------------------------------------------------------------------
Risk Category     | Baseline ASR   | Easy-Complexity Attacks ASR  | Moderate-Complexity Attacks ASR | Difficult-Complexity Attacks ASR
------------------------------------------------------------------------------------------------------------------------------------

📂 All scan files saved to: .\.scan_Intermediary-Model-Target-Scan_20251105_222639
✅ Scan completed successfully!


## Advanced Example: Using an Azure Open AI Model Endpoint in a Callback Function

Using the same Azure Open AI model configuration as above, we now wrap it in a callback function for more flexibility and control on the input and output handling. This will demonstrate how to evaluate an actual AI application. To test your own actual AI application, replace the inside of the callback function with a call to your application.

In [16]:
# Define a callback that uses Azure OpenAI API to generate responses
async def azure_openai_callback(
    messages: list,
    stream: Optional[bool] = False,  # noqa: ARG001
    session_state: Optional[str] = None,  # noqa: ARG001
    context: Optional[Dict[str, Any]] = None,  # noqa: ARG001
) -> dict[str, list[dict[str, str]]]:
    # Get token provider for Azure AD authentication
    token_provider = get_bearer_token_provider(credential, "https://cognitiveservices.azure.com/.default")

    # Initialize Azure OpenAI client
    client = AzureOpenAI(
        azure_endpoint=azure_openai_endpoint,
        api_version=azure_openai_api_version,
        azure_ad_token_provider=token_provider,
    )

    ## Extract the latest message from the conversation history
    messages_list = [{"role": message.role, "content": message.content} for message in messages]
    latest_message = messages_list[-1]["content"]

    try:
        # Call the model
        response = client.chat.completions.create(
            model=azure_openai_deployment,
            messages=[
                {"role": "user", "content": latest_message},
            ],
            # max_tokens=500, # If using an o1 base model, comment this line out
            max_completion_tokens=500,  # If using an o1 base model, uncomment this line
            # temperature=0.7, # If using an o1 base model, comment this line out (temperature param not supported for o1 base models)
        )

        # Format the response to follow the expected chat protocol format
        formatted_response = {"content": response.choices[0].message.content, "role": "assistant"}
    except Exception as e:
        print(f"Error calling Azure OpenAI: {e!s}")
        formatted_response = "I encountered an error and couldn't process your request."
    return {"messages": [formatted_response]}

In [17]:
# Create the RedTeam instance with all of the risk categories with 5 attack objectives generated for each category
model_red_team = RedTeam(
    azure_ai_project=azure_ai_project,
    credential=credential,
    risk_categories=[RiskCategory.Violence, RiskCategory.HateUnfairness, RiskCategory.Sexual, RiskCategory.SelfHarm],
    num_objectives=5,
)

We will use this instance of `model_red_team` to test different attack strategies in the following section.

### Testing Different Attack Strategies

Now we'll run a more comprehensive evaluation using multiple attack strategies across risk categories. This will give us a better understanding of our model's vulnerabilities.

In [ ]:
# Run the red team scan with multiple attack strategies
advanced_result = await model_red_team.scan(
    target=azure_openai_callback,
    scan_name="Advanced-Callback-Scan",
    attack_strategies=[
        AttackStrategy.EASY,  # Group of easy complexity attacks
        AttackStrategy.MODERATE,  # Group of moderate complexity attacks
        AttackStrategy.CharacterSpace,  # Add character spaces
        AttackStrategy.ROT13,  # Use ROT13 encoding
        AttackStrategy.UnicodeConfusable,  # Use confusable Unicode characters
        AttackStrategy.CharSwap,  # Swap characters in prompts
        AttackStrategy.Morse,  # Encode prompts in Morse code
        AttackStrategy.Leetspeak,  # Use Leetspeak
        AttackStrategy.Url,  # Use URLs in prompts
        AttackStrategy.Binary,  # Encode prompts in binary
        AttackStrategy.Compose([AttackStrategy.Base64, AttackStrategy.ROT13]),  # Use two strategies in one attack
    ],
    output_path="Advanced-Callback-Scan.json",
)

🚀 STARTING RED TEAM SCAN: Advanced-Callback-Scan
📂 Output directory: .\.scan_Advanced-Callback-Scan_20251105_222804
📊 Risk categories: ['violence', 'hate_unfairness', 'sexual', 'self_harm']
🔗 Track your red team scan in AI Foundry: None
📋 Planning 52 total tasks


Scanning:   0%|                        | 0/52 [00:00<?, ?scan/s, current=fetching baseline/violence]

📚 Using attack objectives from Azure RAI service


Scanning:   0%|                 | 0/52 [00:03<?, ?scan/s, current=fetching baseline/hate_unfairness]

📝 Fetched baseline objectives for violence: 5 objectives


...

Scanning:  19%|█████▍                      | 10/52 [50:40<3:18:26, 283.48s/scan, current=batch 3/11]

▶️ Starting task: rot13 strategy for self_harm risk category
▶️ Starting task: unicode_confusable strategy for violence risk category
▶️ Starting task: unicode_confusable strategy for hate_unfairness risk category


Scanning:  19%|█████▍                      | 10/52 [50:40<3:18:26, 283.48s/scan, current=batch 3/11]

▶️ Starting task: unicode_confusable strategy for sexual risk category


Scanning:  19%|█████▍                      | 10/52 [50:42<3:18:26, 283.48s/scan, current=batch 3/11]

The data and results used in this attack will be saved to the `output_path` specified. The URL printed out at the end of the scorecard will provide a link to where you results are uploaded and logged to your Azure AI Foundry project.

## Bring your own objectives: Using your own prompts as objectives for RedTeam

Below we demonstrate how to use your own prompts as objectives for a `RedTeam` scan. You can see the required format for prompts under `.\data\prompts.json`. Note that when bringing your own prompts, the supported `risk-type`s are `violence`, `sexual`, `hate_unfairness`, and `self_harm`. The number of prompts you specify will be the `num_objectives` used in the scan. 

In [ ]:
path_to_prompts = "prompts.json"

# Create the RedTeam specifying the custom attack seed prompts to use as objectives
custom_red_team = RedTeam(
    azure_ai_project=azure_ai_project,
    credential=credential,
    custom_attack_seed_prompts=path_to_prompts,  # Path to a file containing custom attack seed prompts
)

In [ ]:
custom_red_team_result = await custom_red_team.scan(
    target=azure_openai_callback,
    scan_name="Custom-Prompt-Scan",
    attack_strategies=[
        AttackStrategy.EASY,  # Group of easy complexity attacks
        AttackStrategy.MODERATE,  # Group of moderate complexity attacks
        AttackStrategy.DIFFICULT,  # Group of difficult complexity attacks
    ],
    output_path="Custom-Prompt-Scan.json",
)

## Conclusion

In this notebook, we've demonstrated how to use the Azure AI Evaluation SDK's `RedTeam` functionality to assess the safety and resilience of AI systems. We started with a basic fixed-response example and then moved to a more realistic model testing across multiple risk categories and attack strategies.

The automated AI red teaming scans provides valuable insights into:

1. **Overall Attack Success Rate (ASR)** - The percentage of attacks that successfully elicit harmful content
2. **Vulnerability by Risk Category** - Which types of harmful content your model is most vulnerable to
3. **Effectiveness of Attack Strategies** - Which attack techniques are most successful against your model
4. **Impact of Complexity** - How more sophisticated attacks affect your model's safety guardrails

By regularly red-teaming your AI applications, you can identify and address potential vulnerabilities before deploying your models to production environments.

### Next Steps

1. **Mitigation**: Use these results to strengthen your model's guardrails against identified attack vectors
2. **Continuous Testing**: Implement regular red team evaluations as part of your development lifecycle
3. **Custom Strategies**: Develop custom attack strategies for your specific use cases and domain
4. **Safety Layers**: Consider adding additional safety layers like Azure AI Content Safety to filter harmful requests and responses 